<a href="https://colab.research.google.com/github/ptran1203/fxgan/blob/master/notebooks/fxgan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x
from google.colab import drive, output
drive.mount('/content/drive')

!rm -rf '/content/gan_project'
!git clone https://github.com/ptran1203/gan_project
!pip install git+https://www.github.com/keras-team/keras-contrib.git

output.clear()

## Create the link from this drive folder to your drive.

---


https://drive.google.com/drive/folders/1RNJXceXkNatuAbNn-CKB8MrgaEHG5RpM?usp=sharing

In [2]:
cd gan_project

/content/gan_project


In [4]:

# K.common.set_image_dim_ordering('tf')
BASE_DIR = '/content/drive/My Drive/bagan'
DS_DIR = '/content/drive/My Drive/bagan/dataset/chest_xray'
DS_SAVE_DIR = '/content/drive/My Drive/bagan/dataset/save'
gratio_mode = 'uniform'
dratio_mode = 'uniform'

from fxgan import *
from batch_gen import *
from utils import *
from tensorflow.keras.layers import LayerNormalization

def create_dir_if_any(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

class FxGan(FXGAN):
    attribute_loss_weight = 0
    def interval_process(self, epoch):
        if epoch % 400 == 0 and epoch > 0:
            output.clear()

    def build_features_from_d_model(self):
        self.features_from_d_model = Model(
            inputs = self.discriminator.inputs,
            output = self.discriminator.layers[-2].get_output_at(-1),
            name = 'Feature_matching'
        )

    def evaluate_g(self, test_x, test_y): return
    def evaluate_d(self, test_x, test_y): return
    def plot_feature_distr(self, bg, size=500): return

class BatchGen(BatchGenerator):
    to_train_classes = list(range(0, 80))
    # to_train_classes = range(12)
    to_test_classes = list(range(81, 102))


is_test = 0
## Test batch GEN
if is_test:
    bg = BatchGen(BatchGen.TRAIN, 64, 'multi_chest', 64)
    labels = np.array([0, 0, 1, 1, 2, 2, 3 ,1])
    samples = bg.ramdom_kshot_images(4, labels)
    print(samples.shape)

Using TensorFlow backend.


## **Train GAN**

---



In [ ]:
# Stop when ever you want :))
gan_epochs  = 400000
# Discriminator learning rate
adam_lr = 0.00003
# Generator learning rate
g_lr = 0.00001
batch_size = 64
# dataset_name should be "flowers", "ch1est", "multi_chest"
# "chest" is binary classification, "multi_chest" using chest-xray14 dataset
dataset_name = 'multi_chest'
latent_size = 128
# Use resnet architecture for Generator
resnet = False
# Use self-attention mechanism
attention = False

np.random.seed(0)
# Image resoulution
rst = 128
# Number of images used to generate feature for Generator should be 2 - 5
k_shot = 10

# normal: sampling from normal distribution
# code: sampling from latent code distribution (computed by classifier)
sampling = 'normal'
advance_losses = {
    # 'l2_feat': 0.7,
    'triplet': 0.7,
    # 'fm_D': 0.3,
    # 'triplet_D': 0.7,
    # 'recon': 0.5,
}
"""
batch: batch norm
in: instance norm
fn: feature norm
D-fn: use feature norm for Discriminator
"""
norm = 'in-bn-fn'
# Adversarial loss type
loss_type = 'categorical'

is_prune = False

# prune = [1000, 3500] if is_prune else None
prune = [1000] if dataset_name != 'flowers' else []

res_dir =  '/content/drive/My Drive/GAN/result/model_{}_{}_v2'.format(dataset_name,rst)
create_dir_if_any(res_dir)

bg_train_full = BatchGen(BatchGen.TRAIN, batch_size,
                         dataset_name, rst,prune_classes=prune,
                         )

counter = [0] * 12 + [219, 112, 38]
prune = [counter[i] - k_shot for i in range(len(counter))]
bg_test = BatchGen(BatchGen.TEST, batch_size, dataset_name, rst, prune_classes=[])

channels = bg_train_full.dataset_x[0].shape[-1]

print("input data loaded...")

shape = bg_train_full.dataset_x[0].shape

print('img shape', shape)
classes = bg_train_full.get_label_table()
# Initialize statistics information
gan_train_losses = defaultdict(list)
gan_test_losses = defaultdict(list)

img_samples = defaultdict(list)

# For all possible minority classes.
target_classes = np.array(range(len(classes)))


print('train counters: ', bg_train_full.per_class_count)
print('test counters: ', bg_test.per_class_count)

# Train GAN to balance the data
gan = FxGan(
    target_classes, loss_type,
    adam_lr=adam_lr, latent_size=latent_size,res_dir=res_dir,image_shape=shape,
    g_lr = g_lr,
    norm = norm,
    resnet=resnet,
    dataset=dataset_name,
    attention=attention,
    k_shot=1,
    sampling=sampling,
    advance_losses=advance_losses,
)
gan.train(bg_train_full, bg_test, epochs=gan_epochs)

In [ ]:
gan.backup_point(5)

In [ ]:
scatter_plot(
    np.concatenate([bg_train_full.dataset_x, bg_test.dataset_x]),
    np.concatenate([bg_train_full.dataset_y, bg_test.dataset_y]),
    gan.latent_encoder, 'train','tsne')

In [ ]:
include_test = True

classes = []
for i in bg_train_full.classes:
    classes += [i] * 100

if include_test:
    for i in bg_test.classes:
        classes += [i] * 100


gan.plot_cm_for_G(bg_train_full,
                  bg_test if include_test else None,
                  classes)

In [ ]:
Counter(bg_test.dataset_y)

In [ ]:
from data_augmentation.model import *
x_test_u, x_sp_u, y_test_u, y_sp_u = train_test_split(bg_test.dataset_x, bg_test.dataset_y)
# print(y_test_u.shape, y_sp_u.shape)
# print(Counter(y_test_u), Counter(y_sp_u))
evaluate_model_metric(gan.latent_encoder, ( x_sp_u,y_sp_u), 
                                    x_test_u, y_test_u-np.min(y_test_u) ,
                                    k_shot=5, metric='l2')

Plot data

In [ ]:

test = 1
same_img = False
show = False
test_size = 10
bg = bg_train_full if not test else bg_test

gan.show_samples_for_class(bg, 99 , mode = '10')

Save generative image

In [ ]:
images, labels = gan.gen_augment_data(bg_train_full,bg_test)
show_samples(images[:10])
images = images *127.5+127.5
pickle_save((images, labels), '/content/drive/My Drive/generated/multi_chest/gen_v1_{}shot.pkl'.format(k_shot))

In [ ]:
show_samples((images[::-1][:10] - 127.5) / 127.5 )


## Tu day de kiem tra mo hinh

In [ ]:
%tensorflow_version 1.x
from google.colab import drive, output
drive.mount('/content/drive')

!rm -rf '/content/gan_project'
!git clone https://github.com/ptran1203/gan_project
!pip install git+https://www.github.com/keras-team/keras-contrib.git
!pip install image-classifiers


## for Model definition/training
from keras.models import Model, load_model
from sklearn.model_selection import train_test_split
from keras import applications
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from keras.regularizers import l2
import keras.backend as K
from keras_contrib.applications.resnet import ResNet, basic_block
from keras_contrib.applications.densenet import DenseNet
import keras.applications as k_apps
from keras.layers import (
    Input, Dense, Reshape,
    Flatten, Embedding, Dropout,
    BatchNormalization, Activation,
    Lambda,Layer, Add, Concatenate,
    Average,GlobalAveragePooling2D,
    MaxPooling2D, AveragePooling2D,
)
from keras.layers.convolutional import (
    UpSampling2D, Convolution2D,
    Conv2D, Conv2DTranspose
)
from keras.optimizers import Adam, SGD

from tensorflow.examples.tutorials.mnist import input_data
from keras.utils import to_categorical
import tensorflow as tf 
import numpy as np
import pickle

output.clear()

In [ ]:
cd gan_project

In [ ]:
from classification_models.keras import Classifiers
from tensorflow.examples.tutorials.mnist import input_data
from keras.preprocessing.image import ImageDataGenerator
import keras.preprocessing.image as iprocess
from mlxtend.plotting import plot_confusion_matrix
import keras
import datetime
from triplet_loss import *
from utils import *
from const import *

import metrics
from visualization import *
from data_augmentation.data_loader import load_gen, load_dataset
from data_augmentation.model import *
import sklearn.metrics as sk_metrics


## Load dataset
# load data
np.random.seed(0)
dataset = 'multi_chest'
resolution = 128
train_classes = 15
[
    x_train, y_train, x_val, y_val,
    x_test, y_test , x_unseen, y_unseen
] = load_dataset(dataset, resolution, 0, train_classes)

x_train, y_train = prune(
    x_train, y_train,
    [1000] + [0] * 14
    # [1000]
)
Counter(y_train)

In [ ]:
rst, channel = x_train.shape[2:]
input_shape = (rst, rst, channel)
classes = np.unique(y_train)
num_of_classes = len(classes)
y_train_onehot = to_categorical(y_train, num_of_classes)
y_val_onehot = to_categorical(y_val, num_of_classes)
feat_dims = 128
from_scratch = False

print('Total classes: ', num_of_classes)
print('Test on {} samples'.format(len(y_test)))

frozen_block = []
all_scores = {}
experiments = 5
save = False
opts = [
    Option.gan_v1,
    Option.bagan,
    # Option.vgg16,
    # Option.vgg16_st_aug,
    # Option.gan_v1,
]

feature_extractor = 'vgg16'
learning_rate = 1e-5 if experiments == 1 else 3e-6
loss_weights = [1, 0.0001]
epochs = 15
loss_type=Losses.center
train_model = None
k_shots = [10,5]

for opt in opts:
    for k_shot in k_shots:
        print("\n======= Experiment {} - {} shot".format(model_map[opt], k_shot))
        result, train_model = run(opt, (x_val, y_val),
                    experiments, frozen_block,
                    feature_extractor, save, learning_rate,
                    loss_weights, epochs, loss_type=loss_type,k_shot=k_shot)

        all_scores[model_map[opt]] = result
        send("experiemnts {} {} shot \n {}".format(model_map[opt], k_shot, json.dumps(result.tolist(), indent=4)))

if loss_type == Losses.center and experiments > 1:
    table = metrics.draw_md_table(all_scores)
    print(table)
    # send slack
    send(table)